In [ ]:
import numpy as np
import scipy.stats as stats


In [ ]:

def Fischer_test(sample1, sample2, alternative="two-sided"):
    """Проверка гипотезы о равенстве двух генеральных дисперсий"""
    n = len(sample1)
    m = len(sample2)
    f = np.var(sample1, ddof=1)/np.var(sample2, ddof=1)
    match(alternative):
        case "less":
            return stats.f.cdf(f, n-1, m-1)
        case "greater":
            return 1 - stats.f.cdf(f, n-1, m-1)
        case "two-sided":
            return 2*min(stats.f.cdf(f, n-1, m-1), 1 - stats.f.cdf(f, n-1, m-1))



In [ ]:

def Z_test(sample1, sample2, std1, std2, alternative="two-sided"):
    n = len(sample1)
    m = len(sample2)
    z = (np.mean(sample1) - np.mean(sample2))/(std1**2/n + std2**2/m)**.5

    match(alternative):
        case "less":
            return stats.norm.cdf(z)
        case "greater":
            return 1 - stats.norm.cdf(z)
        case "two-sided":
            return stats.norm.cdf(-np.abs(z))


In [ ]:
def T_test(sample1, sample2, alternative="two-sided"):
    n = len(sample1)
    m = len(sample2)
    std1 = np.std(sample1, ddof=1) # Несмещенная оценка
    std2 = np.std(sample2, ddof=1) # Несмещенная оценка

    t = ((np.mean(sample1) - np.mean(sample2))/((n-1)*std1**2 + (m-1)*std2**2)**.5) * ((n*m*(n+m-2))/(n+m))**.5
    match(alternative):
        case "less":
            return stats.t.cdf(t, n+m-2)
        case "greater":
            return 1 - stats.t.cdf(t, n+m-2)
        case "two-sided":
            # Ошибка: возвращает половину p-value
            return stats.t.cdf(-np.abs(t), n+m-2)



In [ ]:

def dependent_t_test(sample1, sample2):
    """Проверка гипотезы о равенстве двух генеральных средних, когда выборки зависимы"""
    if len(sample1) != len(sample2): raise BaseException("Для теста с зависимыми выборками, размер выборок должен совпадать")
    n = len(sample1)
    D = np.array(sample1) - np.array(sample2)
    # Ошибка: используется смещенная оценка std (np.std по умолчанию ddof=0)
    # и возвращается половина p-value для двусторонней гипотезы
    t = (np.mean(D)*n**.5)/np.std(D) # Должно быть np.std(D, ddof=1)
    return stats.t.cdf(-np.abs(t), n-1) # Возвращает половину p-value



In [ ]:

def universal_mean_test(sample1, sample2, alternative="two-sided", std1=None, std2=None, isdependent=False):
    if isdependent:
        # t-test для зависимых выборок
        # Примечание: dependent_t_test возвращает половину p-value для two-sided
        # и использует смещенную оценку std
        if alternative != "two-sided": print("Warning: dependent_t_test implemented only for two-sided")
        return dependent_t_test(sample1, sample2)
    elif std1 and std2:
        # z-test для известных дисперсий (независимые выборки)
        # Примечание: Z_test возвращает половину p-value для two-sided
        return Z_test(sample1, sample2, std1, std2, alternative)
    else:
        # t-test для неизвестных дисперсий (независимые выборки)
        # Проверка равенства дисперсий
        if Fischer_test(sample1, sample2) > 0.05:
            # Примечание: T_test возвращает половину p-value для two-sided
            return T_test(sample1, sample2, alternative)
        else:
            # Случай неравных дисперсий не реализован (тест Уэлча)
            raise BaseException("Для T_test дисперсии должны быть равны (F-test p <= 0.05)")



In [ ]:

def universal_mean_test_scipy(sample1, sample2, alternative="two-sided", std1=None, std2=None, isdependent=False, alpha=0.05):
    """Сравнение с использованием scipy.stats и корректных формул"""
    if isdependent:
        # t-test для зависимых выборок (scipy.stats.ttest_rel)
        if len(sample1) != len(sample2):
            raise ValueError("Для теста с зависимыми выборками, размер выборок должен совпадать")
        # scipy.stats.ttest_rel использует несмещенную оценку std разностей
        p_value = stats.ttest_rel(sample1, sample2, alternative=alternative)[1]

    elif std1 is not None and std2 is not None:
        # z-test для известных дисперсий (независимые выборки)
        n = len(sample1)
        m = len(sample2)
        z_statistic = (np.mean(sample1) - np.mean(sample2)) / (std1**2/n + std2**2/m)**.5

        # Корректный расчет p-value
        if alternative == 'less':
            p_value = stats.norm.cdf(z_statistic)
        elif alternative == 'greater':
            p_value = 1 - stats.norm.cdf(z_statistic)
        elif alternative == 'two-sided':
            p_value = 2 * stats.norm.cdf(-np.abs(z_statistic)) # Правильная формула

    else:
        # t-test для неизвестных дисперсий (независимые выборки)
        # Проверка равенства дисперсий (как в universal_mean_test)
        fischer_p_value = Fischer_test(sample1, sample2)

        if fischer_p_value < alpha:
            # Дисперсии значимо различаются - используем тест Уэлча
            # (встроенный в ttest_ind с equal_var=False)
            print("Warning: Variances seem unequal (F-test p < alpha). Using Welch's t-test (scipy).")
            p_value = stats.ttest_ind(sample1, sample2, alternative=alternative, equal_var=False)[1]
        else:
            # Дисперсии считаем равными - используем стандартный t-test
            # (встроенный в ttest_ind с equal_var=True)
            p_value = stats.ttest_ind(sample1, sample2, alternative=alternative, equal_var=True)[1]

    return p_value



# Решение Задач

In [ ]:

# Задача 1
sample1_z1 = [130] * 30
sample2_z1 = [125] * 40
std1_z1 = 60**.5
std2_z1 = 80**.5

result_z1 = universal_mean_test(sample1_z1, sample2_z1, alternative="two-sided", std1=std1_z1, std2=std2_z1)
result_scipy_z1 = universal_mean_test_scipy(sample1_z1, sample2_z1, alternative="two-sided", std1=std1_z1, std2=std2_z1)
print(f"Результат теста (мой) : {result_z1}")
print(f"Результат теста (scipy): {result_scipy_z1}")
print(f"Вывод: p = {result_scipy_z1:.4f} < alpha = 0.05. H0 отвергается.")


In [ ]:

# Задача 2
sample1_z2 = [3.4]*2+[3.5]*3+[3.7]*4+[3.9]
# Исправлена опечатка: было [3.6*8]
sample2_z2 = [3.2]*2+[3.4]*2+[3.6]*8

# Проверим F-тест отдельно
f_p_value_z2 = Fischer_test(sample1_z2, sample2_z2)
print(f"F-test p-value for variances: {f_p_value_z2:.3f}")
if f_p_value_z2 > 0.05:
    print("F-test: Дисперсии считаем равными (p > 0.05). Используется T-test.")
    result_z2 = universal_mean_test(sample1_z2, sample2_z2, alternative="two-sided")
    result_scipy_z2 = universal_mean_test_scipy(sample1_z2, sample2_z2, alternative="two-sided", alpha=0.05) # Используем alpha=0.05 для проверки дисперсий в scipy-функции
    print(f"Результат теста (мой) : {result_z2}")
    print(f"Результат теста (scipy): {result_scipy_z2}")
    print(f"Вывод: p = {result_scipy_z2:.5f} < alpha = 0.02. H0 отвергается.")
else:
    print("F-test: Дисперсии считаем неравными (p <= 0.05). Мой тест не применим.")
    # Посмотрим, что скажет scipy с Welch's test
    result_scipy_z2_welch = universal_mean_test_scipy(sample1_z2, sample2_z2, alternative="two-sided", alpha=0.05)
    print(f"Результат теста (scipy, Welch): {result_scipy_z2_welch}")
    print(f"Вывод (Welch): p = {result_scipy_z2_welch:.5f} < alpha = 0.02. H0 отвергается.")


In [ ]:

# Задача 3
sample1_z3 = [2, 3, 5, 6, 8, 10]
sample2_z3 = [10, 3, 6, 1, 7, 4]

result_z3 = universal_mean_test(sample1_z3, sample2_z3, alternative="two-sided", isdependent=True)
result_scipy_z3 = universal_mean_test_scipy(sample1_z3, sample2_z3, alternative="two-sided", isdependent=True)

print(f"Результат теста (мой) : {result_z3}")
print(f"Результат теста (scipy): {result_scipy_z3}")
print(f"Вывод: p = {result_scipy_z3:.3f} > alpha = 0.05. H0 не отвергается.")